# Query Execution Statistics Analysis using AI Functions 🤖

This notebook analyzes query execution statistics from Databricks System Tables with **AI-powered insights**. Use this as a sample to further enhance functionality

**Data Source:** `system.query.history`

**🤖 AI-Enhanced Analysis:**
- **Intelligent Query Classification** using `ai_classify()`
- **Automated Entity Extraction** using `ai_extract()`
- **Smart Query Summarization** using `ai_summarize()`
- **Predictive Query Performance** using `ai_forecast()`

---

## Setup and Configuration

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
import pandas as pd
from datetime import datetime, timedelta

# Configuration - Adjust as needed
ANALYSIS_DAYS = 30  # Number of days to analyze
TOP_N_QUERIES = 20  # Number of top queries to show
AI_SAMPLE_SIZE = 100  # Number of queries to analyze with AI (for cost control)
USE_AI_FEATURES = True  # Toggle AI features on/off

print(f"Analysis Period: Last {ANALYSIS_DAYS} days")
print(f"Top N Queries: {TOP_N_QUERIES}")
print(f"AI Analysis Sample Size: {AI_SAMPLE_SIZE}")
print(f"AI Features Enabled: {USE_AI_FEATURES}")
print(f"Analysis Run Time: {datetime.now()}")

## Data Loading and Basic Statistics

In [0]:
# Load query history data
query_history_df = spark.table("system.query.history")

# Filter for the analysis period
analysis_start_date = (datetime.now() - timedelta(days=ANALYSIS_DAYS)).strftime('%Y-%m-%d')

filtered_queries = query_history_df.filter(
    f"start_time >= '{analysis_start_date}'"
)

# Cache the filtered dataset for better performance
# filtered_queries.cache()

# Basic statistics
total_queries = filtered_queries.count()
successful_queries = filtered_queries.filter("execution_status = 'FINISHED'").count()
failed_queries = filtered_queries.filter("execution_status = 'FAILED'").count()
canceled_queries = filtered_queries.filter("execution_status = 'CANCELED'").count()

print(f"\n{'='*60}")
print(f"QUERY EXECUTION OVERVIEW (Last {ANALYSIS_DAYS} days)")
print(f"{'='*60}")
print(f"Total Queries:      {total_queries:,}")
print(f"Successful:         {successful_queries:,} ({successful_queries/total_queries*100:.1f}%)")
print(f"Failed:             {failed_queries:,} ({failed_queries/total_queries*100:.1f}%)")
print(f"Canceled:           {canceled_queries:,} ({canceled_queries/total_queries*100:.1f}%)")
print(f"{'='*60}\n")

## 🤖 Query Classification

Using **`ai_classify()`** to intelligently categorize queries by:
- Query complexity (Simple, Medium, Complex)
- Query purpose (Analytics, ETL, Reporting, Exploration)
- Optimization need (Low, Medium, High, Critical)

In [0]:
%sql
-- Create a sample of queries for AI analysis (to control costs)
CREATE OR REPLACE TEMPORARY VIEW ai_analysis_sample AS
SELECT 
  statement_id,
  statement_text,
  statement_type,
  total_duration_ms,
  read_bytes,
  read_rows,
  executed_by,
  start_time
FROM system.query.history
WHERE start_time >= CURRENT_DATE() - INTERVAL 30 DAYS
  AND execution_status = 'FINISHED'
  AND statement_text IS NOT NULL
  AND LENGTH(statement_text) > 50  -- Meaningful queries only
  AND statement_type = 'SELECT'
ORDER BY read_bytes DESC
LIMIT 100;  -- Sample for cost-effective AI analysis

In [0]:
%sql
-- AI-Powered Query Complexity Classification
CREATE OR REPLACE TEMPORARY VIEW ai_query_complexity AS
SELECT 
  statement_id,
  CASE 
    WHEN LENGTH(statement_text) > 200 THEN SUBSTRING(statement_text, 1, 200)
    ELSE statement_text 
  END as query_preview,
  total_duration_ms,
  read_bytes,
  -- Use AI to classify query complexity
  ai_classify(
    statement_text,
    ARRAY('Simple', -- - Single table with basic filters',
          'Medium',--- Multiple tables with joins',
          'Complex',  -- Multiple joins, subqueries, or aggregations',
          'Very Complex' -- Window functions, CTEs, nested subqueries'
    )
  ) as complexity_category,
  -- Use AI to classify query purpose
  ai_classify(
    statement_text,
    ARRAY('Reporting - Dashboard or report query',
          'Analytics - Exploratory data analysis',
          'ETL - Data transformation or loading',
          'Monitoring - System or data quality checks')
  ) as estimated_query_purpose
FROM ai_analysis_sample;

### Use this SQL to find out complexity when you are ready to further fine tune the SQLs for performance Optimizations

In [0]:
%sql
-- View AI classification results
SELECT
  complexity_category,
  estimated_query_purpose,
  COUNT(*) as query_count,
  ROUND(AVG(total_duration_ms)/1000, 2) as avg_duration_seconds,
  ROUND(AVG(read_bytes)/1024/1024/1024, 2) as avg_gb_scanned
FROM ai_query_complexity
GROUP BY complexity_category, estimated_query_purpose
ORDER BY query_count DESC;

## 🧠 Entity Extraction
### This could be very useful to find the candidates for Liquid Clustering 

Using **`ai_extract()`** to automatically identify:
- Table names
- Column names in WHERE clauses
- Join keys
- Filter predicates

In [0]:
%sql
-- Extract key entities from queries using AI
CREATE OR REPLACE TEMPORARY VIEW ai_extracted_entities AS
SELECT 
  statement_id,
  statement_text,
  -- Extract table names
  ai_extract(
    statement_text,
    ARRAY('table_name')
  ) as extracted_tables,
  total_duration_ms,
  read_bytes
FROM ai_analysis_sample
LIMIT 100;  -- Smaller sample for extraction

In [0]:

%sql
-- View extracted entities
WITH T1 AS (
  SELECT 
    --SUBSTRING(statement_text, 1, 150) as query_preview,
    CONCAT_WS('<SQL_END>',COLLECT_LIST(statement_text)) statement_text,
    extracted_tables.table_name as table_names,
    MAX(ROUND(total_duration_ms/1000, 2)) as max_duration_seconds,
    AVG(ROUND(total_duration_ms/1000, 2)) as avg_duration_seconds,
    MAX(ROUND(read_bytes/1024/1024/1024, 2)) as max_gb_scanned
  FROM ai_extracted_entities
  WHERE extracted_tables IS NOT NULL
  GROUP BY extracted_tables.table_name
), T2 AS (
SELECT 
  table_names,
  max_duration_seconds,
  avg_duration_seconds,
  max_gb_scanned,
  -- Extract filter columns
  ai_extract(
    statement_text,
    ARRAY('filter_column', 'join_column')
  ) as extracted_columns
FROM T1
), final as (
SELECT
  T2.table_names,
  T2.extracted_columns.filter_column,
  T2.extracted_columns.join_column,
  --T2.extracted_columns.partition_column,
  T2.max_duration_seconds,
  T2.max_gb_scanned,
  T2.avg_duration_seconds
FROM T2
)
select * from final

## 📝 AI-Powered Query Summarization

Using **`ai_summarize()`** to create human-readable summaries of complex queries

In [0]:
%sql
-- Summarize complex queries for easier understanding
SELECT 
  statement_id,
  ROUND(total_duration_ms/1000, 2) as duration_seconds,
  ROUND(read_bytes/1024/1024/1024, 2) as gb_scanned,
  -- AI-generated query summary
  ai_summarize(statement_text) as query_summary
FROM ai_analysis_sample
WHERE LENGTH(statement_text) > 300  -- Focus on complex queries
ORDER BY read_bytes DESC
LIMIT 10;

## 📈 AI-Powered Query Performance Forecasting

Using **`ai_forecast()`** to predict future query performance trends

In [0]:
%sql
-- Prepare time series data for forecasting
CREATE OR REPLACE TEMPORARY VIEW query_performance_timeseries AS
SELECT 
  DATE(start_time) as query_date,
  COUNT(*) as query_count,
  AVG(total_duration_ms)/1000 as avg_duration_seconds,
  SUM(read_bytes)/1024/1024/1024 as total_gb_scanned,
  AVG(read_bytes)/1024/1024/1024 as avg_gb_per_query
FROM system.query.history
WHERE start_time >= CURRENT_DATE() - INTERVAL 30 DAYS
  AND execution_status = 'FINISHED'
GROUP BY DATE(start_time)
ORDER BY query_date;

In [0]:
%sql
-- View historical trends
SELECT 
  query_date,
  query_count,
  ROUND(avg_duration_seconds, 2) as avg_duration_seconds,
  ROUND(total_gb_scanned, 2) as total_gb_scanned,
  ROUND(avg_gb_per_query, 2) as avg_gb_per_query
FROM query_performance_timeseries
ORDER BY query_date DESC
LIMIT 30;

Databricks visualization. Run in Databricks to view.